In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial import distance 
import seaborn as sns

In [2]:
#get data with content, embeddings, and named entities 
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/dataWithEmbeddings.tsv", sep="\t", converters={"embedding":lambda x: np.array(x.strip("[]").split(), dtype=float)})

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,7,10,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(df.shape)
df["key"] = df["key"].apply(lambda x: x.strip())
print(df.drop_duplicates(subset=["key"]).shape)

(308837, 48)
(308837, 48)


In [4]:
print(max(pd.to_datetime(df["date"])))
print(min(pd.to_datetime(df["date"])))
print(df.shape)

2021-09-01 00:00:00
2021-05-01 00:00:00
(308837, 48)


In [5]:
testDf = df[["key", "topics", "embedding"]]

print("parsing")

def cleanList(inList): 
    return [str(re.sub("[^a-zA-Z0-9 ]", "", item).lower()) for item in inList]

def parseList(inStr): 
    split = inStr.split("\'), (\'")
    return [cleanList(item.split("', '")) for item in split]

#parse topics from string to actual list of tuples 
testDf["topics"] = testDf["topics"].apply(parseList)

print("parsed")

#test out idea for creating reverse mapping 
testDf = testDf.dropna(subset=["topics"])

#bring each tuple into its own row 
testDf = testDf.explode("topics")

#bring each tuple entry into its own column 
#split ent_type, entity pairs to columns 
testDf[["ent_type","entity"]] = pd.DataFrame(testDf["topics"].tolist(), index=testDf.index)

print("formatted") 

#keep only the entity types that may be interesting 
toKeep = ["org","event", "person", "work_of_art", "product"]
testDf = testDf[testDf["ent_type"].isin(toKeep)]

grouped = testDf[["embedding", "ent_type", "entity", "key"]].groupby(by=["ent_type", "entity"]).agg(list)

parsing


<ipython-input-5-80b088d75f01>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDf["topics"] = testDf["topics"].apply(parseList)


parsed
formatted


In [6]:
grouped["articleNum"] = grouped["key"].apply(len)
grouped.head()

embedding  \
ent_type entity                                                                 
event     fields2value1970  [[-0.00773029588, -0.00490123546, 0.0179314595...   
          game 2            [[-0.0185301453, 0.0250518899, 0.0244161058, -...   
          game of thrones   [[-0.0230182633, 0.0624324679, 0.00821057428, ...   
          hall of fame      [[-0.0627424344, 0.0551135093, 0.00766856782, ...   
          mars 2020         [[-0.0456677824, -0.00853861216, 0.0276035238,...   

                                                                          key  \
ent_type entity                                                                 
event     fields2value1970  [hillsborofreepress_Infinity_Fitness_has_offic...   
          game 2            [theseattletimes_76ers’_Rivers_seeks_fill-in_f...   
          game of thrones   [thenewyorktimes_Seth_Meyers_Explains_How_‘For...   
          hall of fame      [dailydemocrat_49ers’_Hall_of_Fame:_John_Taylo...   
          mars 2020                   [cbsnews_How_NASA_flew_a_drone_on_Mars]   

                            articleNum  
ent_type entity                         
event     fields2value1970           6  
          game 2                     1  
          game of thrones            1  
          hall of fame               3  
          mars 2020                  1

In [116]:
groupedLean = grouped[(grouped["articleNum"] > 5) & (grouped["articleNum"] < 1000)]
groupedLean.shape

(113424, 3)

In [649]:
groupedLean

embedding  \
ent_type entity                                                                      
event     fields2value1970       [[-0.00773029588, -0.00490123546, 0.0179314595...   
          super bowl             [[-0.0758472532, 0.0570456721, 0.0111176176, -...   
         5a championship series  [[-0.059961386, 0.0440411158, -0.0295248628, 0...   
         911                     [[-0.0297710318, 0.00756559148, 0.00346088805,...   
         a cold war              [[-0.0481121317, 0.0955681279, 0.00663438067, ...   
...                                                                            ...   
product  ziploc                  [[-0.0109394677, 0.0796035081, -0.00845626649,...   
         zoom                    [[-0.0750853047, 0.0770440996, -0.0180662386, ...   
         zoom and                [[-0.0261818618, 0.0623469502, 0.0101112816, -...   
         zouk                    [[-0.0174199827, 0.000472159358, -0.0066477721...   
         zumba                   [[-0.000552972138, 0.00241767056, 0.0020816833...   

                                                                               key  \
ent_type entity                                                                      
event     fields2value1970       [hillsborofreepress_Infinity_Fitness_has_offic...   
          super bowl             [theenterpriserecord_Five_things_Patrick_Willi...   
         5a championship series  [aurorasentinel_Baseball:_2021_Class_5A_region...   
         911                     [deseretnews_How_pride_in_2002_Winter_Olympics...   
         a cold war              [thedailyrepublic_Army’s_roots_reach_back_to_n...   
...                                                                            ...   
product  ziploc                  [thetimescourier_East_Ellijay_traffic_stop_res...   
         zoom                    [theenterpriserecord_OLLI_hosting_summer_class...   
         zoom and                [summitdailynews_Wildfire_season_is_here:_Are_...   
         zouk                    [lasvegasreviewjournal_Madison_Beer,_DJ_Snake,...   
         zumba                   [thedailyrepublic_Zumba_marks_20_years_of_danc...   

                                 articleNum  
ent_type entity                              
event     fields2value1970                6  
          super bowl                      9  
         5a championship series           7  
         911                             11  
         a cold war                      23  
...                                     ...  
product  ziploc                           7  
         zoom                           431  
         zoom and                         6  
         zouk                             9  
         zumba                           48  

[112774 rows x 3 columns]

In [80]:
def getPairwise(inList): 
    return [(item1, item2) for item2 in inList for item1 in inList]


#work with a small df first 
testDf = groupedLean[["key", "embedding"]].head(2000)

testDf["embedding"] = testDf["embedding"].apply(getPairwise)
testDf["key"] = testDf["key"].apply(getPairwise)

exploded = testDf.apply(pd.Series.explode).drop_duplicates(subset=["key"])
testDf.head(3)


key  \
ent_type entity                                                                      
event     fields2value1970       [(hillsborofreepress_Infinity_Fitness_has_offi...   
          super bowl             [(theenterpriserecord_Five_things_Patrick_Will...   
         5a championship series  [(aurorasentinel_Baseball:_2021_Class_5A_regio...   

                                                                         embedding  
ent_type entity                                                                     
event     fields2value1970       [([-0.00773029588, -0.00490123546, 0.017931459...  
          super bowl             [([-0.0758472532, 0.0570456721, 0.0111176176, ...  
         5a championship series  [([-0.059961386, 0.0440411158, -0.0295248628, ...

In [82]:
exploded.shape

(4424296, 2)

In [104]:
from tqdm.notebook import tqdm 
from multiprocessing import Pool 

def getCos(inList): 
    return 1 - distance.cosine(inList[0], inList[1])

def getCosSeries(inSeries): 
    return inSeries.apply(getCos)

#tqdm.pandas()
#exploded.head(1000000)["embedding"].progress_map(getCos)

testDf = exploded
testEmbeddings = testDf["embedding"]

with Pool(12) as pool: 
    splitList = np.array_split(testEmbeddings, 10)
    similarityArrs = list(tqdm(pool.imap(getCosSeries, splitList), total=10))
    similarity = pd.concat(similarityArrs)

testDf["similarity"] = similarity

  0%|          | 0/10 [00:02<?, ?it/s]

In [112]:
sanity = testDf[testDf["similarity"] >= .7]
sanity["key"][203]

("dailyherald_'Pure_business'_at_Biden-Putin_summit:_No_hugs,_no_brickbats",
 'waterburyrepublicanamerican_‘Pure_business’_at_Biden-Putin_summit:_No_hugs,_no_brickbats')

# Version to run as script on server!

In [13]:
import pandas as pd
import numpy as np
import csv
import re
from scipy.spatial import distance 
from tqdm import tqdm 
from multiprocessing import Pool 

#Output path for the dataframe after filtering and getting cosine similarity of filtered pairs 
outputPath = "/home/blitt/projects/localNews/data/interim/similarityAnalysis/articlePairsCosineSim.tsv"

#read in data 
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/dataWithEmbeddings.tsv", sep="\t", converters={"embedding":lambda x: np.array(x.strip("[]").split(), dtype=float)})

print("Input data shape: " + str(df.shape))

df = df[["key", "topics", "embedding"]]

print("parsing")

def cleanList(inList): 
    return [str(re.sub("[^a-zA-Z0-9 ]", "", item).lower()) for item in inList]

def parseList(inStr): 
    split = inStr.split("\'), (\'")
    return [cleanList(item.split("', '")) for item in split]

#parse topics from string to actual list of tuples 
df["topics"] = df["topics"].apply(parseList)

print("parsed")

#test out idea for creating reverse mapping 
df = df.dropna(subset=["topics"])

#bring each tuple into its own row 
df = df.explode("topics")

#bring each tuple entry into its own column 
#split ent_type, entity pairs to columns 
df[["ent_type","entity"]] = pd.DataFrame(df["topics"].tolist(), index=df.index)

print("formatted") 

#keep only the entity types that may be interesting 
toKeep = ["org","event", "person", "work_of_art", "product"]
df = df[df["ent_type"].isin(toKeep)]

#the data grouped into entity clusters 
grouped = df[["embedding", "ent_type", "entity", "key"]].groupby(by=["ent_type", "entity"]).agg(list)

#get the cluster length 
grouped["articleNum"] = grouped["key"].apply(len)

#filter out named entities that are too common, since they likely aren't that meaningful/specific 
groupedLean = grouped[(grouped["articleNum"] > 5) & (grouped["articleNum"] < 1000)]

print("Shape after grouping: " + str(groupedLean.shape))

def getPairwise(inList): 
    return [(item1, item2) for item2 in inList for item1 in inList]

#we only needed the articleNum column for filtering 
df = groupedLean[["key", "embedding"]]

#get all of the pairs for a given list of articles and a given list of embeddings 
df["embedding"] = df["embedding"].apply(getPairwise)
df["key"] = df["key"].apply(getPairwise)

#"explode" the pairs into a seperate row (this is when we have a TON of data)
exploded = df.apply(pd.Series.explode).drop_duplicates(subset=["key"])

print("Exploded size: " + str(exploded.shape))

def getCos(inList): 
    return 1 - distance.cosine(inList[0], inList[1])

def getCosSeries(inSeries): 
    return inSeries.apply(getCos)

embeddings = exploded["embedding"]

with Pool(12) as pool: 
    splitList = np.array_split(embeddings, 10)
    similarityArrs = list(tqdm(pool.imap(getCosSeries, splitList), total=10))
    similarity = pd.concat(similarityArrs)

exploded["similarity"] = similarity

print("writing")
exploded[["key", "similarity"]].to_csv(outputPath, sep="\t", quoting=csv.QUOTE_NONNUMERIC)
print("written")

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,7,10,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Input data shape: (308837, 48)
parsing
parsed
formatted
Shape after grouping: (921, 3)
Exploded size: (55110, 2)


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 